In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
rr = pd.read_csv("spsdata/roadrunner.csv")
clean = rr[rr["round"] != "warmup"]

In [ ]:
rr.ID.unique()

In [ ]:
clean

In [ ]:
acceleration = []
jerk = []
costheta = []
labels = []
for i in rr.ID.unique():
    acceleration.append(rr[rr.ID == i]["acc"].mean())
    jerk.append(rr[rr.ID == i]["jerk"].mean())
    costheta.append(rr[rr.ID == i]["costheta"].mean())
    labels.append(rr[rr.ID == i]["mabc_binary_score"].mean())
plt.scatter(acceleration, jerk, c=["red" if label == 0 else "blue" for label in labels])
plt.xlabel("acceleration")
plt.ylabel("jerk")
plt.show()
plt.scatter(costheta, acceleration, c=["red" if label == 0 else "blue" for label in labels])
plt.xlabel("costheta")
plt.ylabel("acceleraion")
plt.show()
plt.scatter(costheta, jerk, c=["red" if label == 0 else "blue" for label in labels])
plt.xlabel("costheta")
plt.ylabel("jerk")
plt.show()